test cases:

1. get the expected output
2. can throw out errors (unfitted, format error, etc.)
3. produce correct (or nearly correct) amount of NAs

In [1]:
import pandas as pd
import numpy as np

In [2]:
%cd /Users/alex/PETsARD

/Users/alex/PETsARD


In [3]:
import PETsARD

In [4]:
def metadata_builder(data):
    metadata = {'metadata_col': None, 'metadata_global': {}}

    metadata['metadata_global']['row_num'] = data.shape[0]
    metadata['metadata_global']['col_num'] = data.shape[1]
    metadata['metadata_global']['na_percentage'] = data.isna().any(axis=1).mean()

    # create type and na_percentage keys and values automatically
    metadata['metadata_col'] = data.dtypes.reset_index(name='dtype')\
        .merge(data.isna().mean(axis=0).reset_index(name='na_percentage'), 
               on='index').set_index('index').to_dict('index')
    
    return metadata

In [5]:
loader = PETsARD.Loader(filepath = '[NHANES] B.csv',
                                header_exist = False,
                                header_names = ['gen','age','race','edu','mar','bmi','dep','pir','gh','mets','qm','dia']
                               )
print(loader.data.head(1))

    gen   age   race       edu      mar        bmi  dep  pir   gh  mets  qm  \
0  Male  62.0  White  Graduate  Married  27.799999    0    0  0.0   0.0  Q2   

   dia  
0    1  


In [6]:
df = loader.data

In [7]:
m = metadata_builder(df)

In [8]:
metadata = {'metadata_col': {
                'gen': {'type': 'categorical', 'na_percentage': 0.0},
                'age': {'type': 'numerical', 'na_percentage': 0.0},
                'race': {'type': 'categorical', 'na_percentage': 0.0},
                'edu': {'type': 'categorical', 'na_percentage': 0.0},
                'mar': {'type': 'categorical', 'na_percentage': 0.0},
                'bmi': {'type': 'numerical', 'na_percentage': 0.0},
                'dep': {'type': 'numerical', 'na_percentage': 0.0},
                'pir': {'type': 'numerical', 'na_percentage': 0.0},
                'gh': {'type': 'numerical', 'na_percentage': 0.0},
                'mets': {'type': 'numerical', 'na_percentage': 0.0},
                'qm': {'type': 'categorical', 'na_percentage': 0.0},
                'dia': {'type': 'numerical', 'na_percentage': 0.0}
                },
             'metadata_global':{
                 'row_num': 4190,
                 'col_num': 12,
                 'na_percentage': 0.0
                }
            }

In [9]:
pm = PETsARD.HyperProcessor(m)

In [10]:
from PETsARD.Processor.Missingist import *
from PETsARD.Processor.Outlierist import *
from PETsARD.Processor.Encoder import *
from PETsARD.Processor.Scaler import *

In [11]:
# df.loc[[1,2,3,4,5], 'gen'] = np.nan
# df.loc[[10,20,30,40,5], 'age'] = np.nan

In [12]:
# test global outlierist transformation

config_1 = {'missingist': {'gen': Missingist_Drop(), 'age': Missingist_Mean()},
            'outlierist': {'gen': None, 'age': Outlierist_ZScore()},
            'encoder': {'gen': Encoder_Label()},
            'scaler': {'age': Scaler_Log()}}

In [13]:
pm.update_config(config_1)

In [14]:
pm.get_config()

{'missingist': {'gen': <PETsARD.Processor.Missingist.Missingist_Drop at 0x1580f88e0>,
  'age': <PETsARD.Processor.Missingist.Missingist_Mean at 0x1580fa590>,
  'race': <PETsARD.Processor.Missingist.Missingist_Drop at 0x1580fb070>,
  'edu': <PETsARD.Processor.Missingist.Missingist_Drop at 0x1580f9900>,
  'mar': <PETsARD.Processor.Missingist.Missingist_Drop at 0x1072472e0>,
  'bmi': <PETsARD.Processor.Missingist.Missingist_Mean at 0x1580f9a20>,
  'dep': <PETsARD.Processor.Missingist.Missingist_Mean at 0x1580fb6a0>,
  'pir': <PETsARD.Processor.Missingist.Missingist_Mean at 0x1580fb7c0>,
  'gh': <PETsARD.Processor.Missingist.Missingist_Mean at 0x1580fb460>,
  'mets': <PETsARD.Processor.Missingist.Missingist_Mean at 0x1580fb220>,
  'qm': <PETsARD.Processor.Missingist.Missingist_Drop at 0x1580fb190>,
  'dia': <PETsARD.Processor.Missingist.Missingist_Mean at 0x1580f9c00>},
 'outlierist': {'gen': None,
  'age': <PETsARD.Processor.Outlierist.Outlierist_ZScore at 0x1580af8e0>,
  'race': None,
  

In [15]:
pm.fit(df)

In [16]:
df_transformed = pm.transform(df)

In [17]:
df_inverse = pm.inverse_transform(df_transformed)

/Users/alex/PETsARD/PETsARD/HyperProcessor/HyperProcessor.py:467: RuntimeWarning: invalid value encountered in scalar divide
  adjusted_na_percentage = self._metadata['metadata_col'][col].get('na_percentage', 0.0)\


In [18]:
df_inverse

,gen,age,race,edu,mar,bmi,dep,pir,gh,mets,qm,dia
0,Female,72.000000,Mexican,11th,Separated,28.6,0.0,2.775558e-17,0.0,0.0,Q1,2.775558e-17
1,Male,46.000000,White,Graduate,Parther,27.6,0.0,2.775558e-17,0.0,0.0,Q3,2.775558e-17
2,Male,45.000004,Other,11th,Never,24.1,0.0,2.775558e-17,0.0,0.0,Q3,2.775558e-17
3,Female,30.000002,Hispanic,College,Parther,26.6,0.0,2.775558e-17,0.0,0.0,Q4,2.775558e-17
4,Female,27.000002,Black,College,Never,38.0,0.0,2.775558e-17,0.0,0.0,Q1,2.775558e-17
...,...,...,...,...,...,...,...,...,...,...,...,...
2144,Female,41.000000,Other,Graduate,Married,20.9,0.0,2.775558e-17,0.0,0.0,Q1,2.775558e-17
2145,Male,34.000000,Black,Graduate,Married,30.9,0.0,2.775558e-17,0.0,0.0,Q3,2.775558e-17
2146,Male,25.000002,Other,Graduate,Never,21.0,0.0,2.775558e-17,0.0,0.0,Q1,2.775558e-17
2147,Female,80.000000,White,College,Widowed,31.0,0.0,2.775558e-17,0.0,0.0,Q1,2.775558e-17


In [19]:
try:
    print(f'Inverse successful:\n{(df_inverse == df).all()}')
    print(f'Numeric close (age): {np.isclose(df_inverse.age, df.age).all()}')
    print(f'Numeric close (bmi): {np.isclose(df_inverse.bmi, df.bmi).all()}')
except:
    print('Not applicable.')

Not applicable.


In [20]:
loader2 = PETsARD.Loader(filepath = '[Adt Income] adult.csv'
                               ,na_values = {k : '?' for k in ['workclass'
                                                              ,'occupation'
                                                              ,'native-country'
                                                              ]
                                            }
                               )
print(loader2.data.head(1))

   age workclass  fnlwgt education  educational-num marital-status  \
0   25   Private  226802      11th                7  Never-married   

          occupation relationship   race gender  capital-gain  capital-loss  \
0  Machine-op-inspct    Own-child  Black   Male             0             0   

   hours-per-week native-country income  
0              40  United-States  <=50K  


In [21]:
df2 = loader2.data

In [22]:
metadata2 = {'metadata_col': {
                'age': {'type': 'numerical', 'na_percentage': 0.0},
                'workclass': {'type': 'categorical', 'na_percentage': 0.057307},
                'fnlwgt': {'type': 'numerical', 'na_percentage': 0.0},
                'education': {'type': 'categorical', 'na_percentage': 0.0},
                'educational-num': {'type': 'numerical', 'na_percentage': 0.0},
                'marital-status': {'type': 'categorical', 'na_percentage': 0.0},
                'occupation': {'type': 'categorical', 'na_percentage': 0.057512},
                'relationship': {'type': 'categorical', 'na_percentage': 0.0},
                'race': {'type': 'categorical', 'na_percentage': 0.0},
                'gender': {'type': 'categorical', 'na_percentage': 0.0},
                'capital-gain': {'type': 'numerical', 'na_percentage': 0.0},
                'capital-loss': {'type': 'numerical', 'na_percentage': 0.0},
                'hours-per-week': {'type': 'numerical', 'na_percentage': 0.0},
                'native-country': {'type': 'categorical', 'na_percentage': 0.017546},
                'income': {'type': 'categorical', 'na_percentage': 0.0}
                },
             'metadata_global':{
                 'row_num': 48842,
                 'col_num': 15,
                 'na_percentage': 0.07411653904426518
                }
            }

In [23]:
m2 = metadata_builder(df2)

In [24]:
pm2 = PETsARD.HyperProcessor(m2)

In [25]:
pm2.get_config()

{'missingist': {'age': <PETsARD.Processor.Missingist.Missingist_Mean at 0x15829eb30>,
  'workclass': <PETsARD.Processor.Missingist.Missingist_Drop at 0x15829fa60>,
  'fnlwgt': <PETsARD.Processor.Missingist.Missingist_Mean at 0x15829c730>,
  'education': <PETsARD.Processor.Missingist.Missingist_Drop at 0x15829e5f0>,
  'educational-num': <PETsARD.Processor.Missingist.Missingist_Mean at 0x15829f190>,
  'marital-status': <PETsARD.Processor.Missingist.Missingist_Drop at 0x15829e830>,
  'occupation': <PETsARD.Processor.Missingist.Missingist_Drop at 0x15829ee90>,
  'relationship': <PETsARD.Processor.Missingist.Missingist_Drop at 0x15829e2c0>,
  'race': <PETsARD.Processor.Missingist.Missingist_Drop at 0x15829dab0>,
  'gender': <PETsARD.Processor.Missingist.Missingist_Drop at 0x15829ebf0>,
  'capital-gain': <PETsARD.Processor.Missingist.Missingist_Mean at 0x15829f310>,
  'capital-loss': <PETsARD.Processor.Missingist.Missingist_Mean at 0x15829ecb0>,
  'hours-per-week': <PETsARD.Processor.Missing

In [26]:
pm2.fit(df2)

In [27]:
df_transformed2 = pm2.transform(df2)

In [28]:
df_transformed2

,age,workclass,fnlwgt,education,educational-num,marital-status,occupation,relationship,race,gender,capital-gain,capital-loss,hours-per-week,native-country,income
0,-0.995129,0.200744,0.351675,0.823446,-1.197259,0.520277,0.780175,0.711411,0.886704,0.015783,-0.144804,-0.217127,-0.034087,0.414545,0.597385
1,-0.046942,0.628544,-0.945524,0.168417,-0.419335,0.118204,0.939796,0.300718,0.424883,0.581290,-0.144804,-0.217127,0.772930,0.835362,0.352802
2,-0.776316,0.865460,1.394723,0.845388,0.747550,0.202660,0.990309,0.002062,0.397141,0.307549,-0.144804,-0.217127,-0.034087,0.653678,0.799108
3,-1.068066,0.683043,1.704525,0.341146,-0.030373,0.670634,0.661966,0.851322,0.585355,0.751192,-0.144804,-0.217127,-0.034087,0.759405,0.079847
4,-0.192816,0.982412,0.215911,0.665718,1.136512,0.289358,0.431145,0.335814,0.112585,0.632754,-0.144804,-0.217127,-0.034087,0.195899,0.751762
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26928,1.047121,0.104603,1.251867,0.754273,1.525474,0.408701,0.294089,0.253773,0.747924,0.476418,-0.144804,-0.217127,-0.034087,0.772154,0.784964
26929,-1.213941,0.514112,1.140952,0.436682,-0.030373,0.548471,0.992229,0.506640,0.366696,0.241868,-0.144804,-0.217127,-0.034087,0.789231,0.327293
26930,-0.849254,0.291649,0.640492,0.869339,0.747550,0.169015,0.953609,0.957186,0.688141,0.671006,-0.144804,-0.217127,-0.195490,0.452191,0.650814
26931,0.098933,0.261430,-0.334178,0.207703,-0.419335,0.404841,0.757262,0.059159,0.378882,0.242052,-0.144804,-0.217127,-0.034087,0.165984,0.864951


In [29]:
df_inverse2 = pm2.inverse_transform(df_transformed2)

In [30]:
df_inverse2

,age,workclass,fnlwgt,education,educational-num,marital-status,occupation,relationship,race,gender,capital-gain,capital-loss,hours-per-week,native-country,income
0,25.0,Private,226802.0,11th,7.0,Never-married,Machine-op-inspct,Own-child,Black,Male,0.0,0.0,40.0,United-States,<=50K
1,38.0,Private,89814.0,HS-grad,9.0,Married-civ-spouse,Farming-fishing,Husband,White,Male,0.0,0.0,50.0,United-States,<=50K
2,28.0,Local-gov,336951.0,Assoc-acdm,12.0,Married-civ-spouse,Protective-serv,Husband,White,Male,0.0,0.0,40.0,United-States,>50K
3,24.0,Private,369667.0,Some-college,10.0,Never-married,Other-service,Unmarried,White,Female,0.0,0.0,40.0,United-States,<=50K
4,36.0,Federal-gov,212465.0,Bachelors,13.0,Married-civ-spouse,Adm-clerical,Husband,White,Male,0.0,0.0,40.0,United-States,<=50K
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26928,53.0,Private,321865.0,Masters,14.0,Married-civ-spouse,Exec-managerial,Husband,White,Male,0.0,0.0,40.0,United-States,>50K
26929,22.0,Private,310152.0,Some-college,10.0,Never-married,Protective-serv,Not-in-family,White,Male,0.0,0.0,40.0,United-States,<=50K
26930,27.0,Private,257302.0,Assoc-acdm,12.0,Married-civ-spouse,Tech-support,Wife,White,Female,0.0,0.0,38.0,United-States,<=50K
26931,40.0,Private,154374.0,HS-grad,9.0,Married-civ-spouse,Machine-op-inspct,Husband,White,Male,0.0,0.0,40.0,United-States,>50K


In [31]:
df2

,age,workclass,fnlwgt,education,educational-num,marital-status,occupation,relationship,race,gender,capital-gain,capital-loss,hours-per-week,native-country,income
0,25,Private,226802,11th,7,Never-married,Machine-op-inspct,Own-child,Black,Male,0,0,40,United-States,<=50K
1,38,Private,89814,HS-grad,9,Married-civ-spouse,Farming-fishing,Husband,White,Male,0,0,50,United-States,<=50K
2,28,Local-gov,336951,Assoc-acdm,12,Married-civ-spouse,Protective-serv,Husband,White,Male,0,0,40,United-States,>50K
3,44,Private,160323,Some-college,10,Married-civ-spouse,Machine-op-inspct,Husband,Black,Male,7688,0,40,United-States,>50K
4,18,NaN,103497,Some-college,10,Never-married,NaN,Own-child,White,Female,0,0,30,United-States,<=50K
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48837,27,Private,257302,Assoc-acdm,12,Married-civ-spouse,Tech-support,Wife,White,Female,0,0,38,United-States,<=50K
48838,40,Private,154374,HS-grad,9,Married-civ-spouse,Machine-op-inspct,Husband,White,Male,0,0,40,United-States,>50K
48839,58,Private,151910,HS-grad,9,Widowed,Adm-clerical,Unmarried,White,Female,0,0,40,United-States,<=50K
48840,22,Private,201490,HS-grad,9,Never-married,Adm-clerical,Own-child,White,Male,0,0,20,United-States,<=50K


In [32]:
df_inverse2.isna().mean(axis=0)

age                0.000000
workclass          0.057290
fnlwgt             0.000000
education          0.000000
educational-num    0.000000
marital-status     0.000000
occupation         0.057476
relationship       0.000000
race               0.000000
gender             0.000000
capital-gain       0.000000
capital-loss       0.000000
hours-per-week     0.000000
native-country     0.017525
income             0.000000
dtype: float64

In [33]:
df_inverse2.isna().any(axis=1).mean()

0.07106523595589054

In [34]:
pm.get_changes()

,processor,col,current,default
0,outlierist,age,Outlierist_ZScore,Outlierist_IQR
1,encoder,gen,Encoder_Label,Encoder_Uniform
2,scaler,age,Scaler_Log,Scaler_Standard
